In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import NamedTupleCursor

conn = psycopg2.connect(database='finance', cursor_factory=NamedTupleCursor)

with conn.cursor() as cur:
    cur.execute('''
    WITH inflows AS (SELECT SUM(total) FROM transactions WHERE target = 'Cash'),
        outflows AS (SELECT SUM(total) FROM transactions WHERE source = 'Cash')
    SELECT inflows.sum - outflows.sum AS cash FROM inflows CROSS JOIN outflows;
    ''')
    cash = cur.fetchone().cash
    
with conn.cursor() as cur:
    cur.execute('''
    SELECT target AS ticker, SUM(units) FROM transactions WHERE txtype = 'buy' GROUP BY ticker ORDER BY ticker ASC;
    ''')
    units = cur.fetchall()
    
tickers = '+'.join([x.ticker for x in units])
prices = pd.read_csv('http://finance.yahoo.com/d/quotes.csv?s=' + tickers + '&f=sl1p', header=None)  # l1 rather than b
prices.set_index(0, inplace=True)  

from decimal import Decimal
previous = cash
total = cash
for asset in units:
    asset_value = asset.sum * Decimal(prices.loc[asset.ticker][1])
    prev_asset_value = asset.sum * Decimal(prices.loc[asset.ticker][2])
    total += asset_value
    previous += prev_asset_value
    day = asset_value - prev_asset_value
    pctg = 100 * day / prev_asset_value
    print(f"{asset.ticker:8}: {asset_value:10,.2f}  ({day:+10,.2f}, {pctg:+6.2f}%)")

day = total - previous
pctg = 100 * day / previous
print(f"Cash    : {cash:10,.2f}")
print(f"Total   : {total:10,.2f}  ({day:+10,.2f}, {pctg:+6.2f}%)")

with conn.cursor() as cur:
    cur.execute('''SELECT SUM(total) AS deposits FROM transactions WHERE txtype = 'deposit';''')
    deposits = cur.fetchone().deposits
    profit = total - deposits
    profit_pctg = 100 * profit / deposits

print()
print(f"Deposits: {deposits:10,.2f}  ({profit:+10,.2f}, {profit_pctg:+6.2f}%)")
